<a href="https://colab.research.google.com/github/nannthd/project_AIEngineer/blob/main/vector_drug_ViTModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/drug/data 50 class add_augment.zip'

#image2vector

##224*224

###50cls

In [ ]:
# use VIT model to convert data
import pandas as pd
import numpy as np
import os
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import torch

# ใช้โมเดล ViT ของ Hugging Face ในการทำ embedding
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

def image_embedding(path):
    img = Image.open(path).convert('RGB').resize((224, 224))  # ปรับขนาดภาพให้เข้ากับ ViT
    inputs = processor(images=img, return_tensors="pt")
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    avg_embedding = last_hidden_states.mean(dim=1).squeeze().detach().numpy()  # ค่าเฉลี่ยของ hidden states

    curr_df = pd.DataFrame(avg_embedding).T
    return curr_df

def process_images_in_folder(folder_path, output_csv_path):
    pdEmbedded = pd.DataFrame()
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

    for idx, image_file in enumerate(image_files):
        image_path = os.path.join(folder_path, image_file)
        embedded = image_embedding(image_path)
        embedded['ID'] = image_file  # ใช้ชื่อไฟล์เป็น ID
        pdEmbedded = pd.concat([pdEmbedded, embedded], ignore_index=True)

    # สร้างโฟลเดอร์สำหรับ output_csv_path
    output_folder = os.path.dirname(output_csv_path)
    os.makedirs(output_folder, exist_ok=True)

    # บันทึก DataFrame ลงในไฟล์ CSV
    pdEmbedded.to_csv(output_csv_path, index=False)

# ฟังก์ชันสำหรับการวนลูปโฟลเดอร์
def process_all_folders(base_folder_path, output_base_folder):
    subfolders = [f.path for f in os.scandir(base_folder_path) if f.is_dir()]

    for subfolder in subfolders:
        subfolder_name = os.path.basename(subfolder)
        output_csv_path = os.path.join(output_base_folder, f"{subfolder_name}.csv")
        process_images_in_folder(subfolder, output_csv_path)
        print(f"Embedding data for {subfolder_name} has been saved to {output_csv_path}")

# ตัวอย่างการใช้งาน
base_folder_path = "/content/data 50 class add_augment - Copy"
output_base_folder = "/content/drug50cls_ViTModel"
os.makedirs(output_base_folder, exist_ok=True)  # สร้างโฟลเดอร์หลักถ้ายังไม่มี

process_all_folders(base_folder_path, output_base_folder)

###Cosine similarity

In [23]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import torch

# ใช้โมเดล ViT ของ Hugging Face ในการทำ embedding
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

# ย้ายโมเดลไปยัง GPU ถ้ามี
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def image_embedding(path):
    try:
        img = Image.open(path).convert('RGB').resize((224, 224))  # ปรับขนาดภาพให้เข้ากับ ViT
        inputs = processor(images=img, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        last_hidden_states = outputs.last_hidden_state
        avg_embedding = last_hidden_states.mean(dim=1).squeeze().cpu().detach().numpy()  # ค่าเฉลี่ยของ hidden states
        curr_df = pd.DataFrame(avg_embedding).T
        return curr_df
    except Exception as e:
        print(f"Error processing image {path}: {e}")
        return pd.DataFrame()

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            # ดรอปคอลัมน์ ID และเก็บเฉพาะเวกเตอร์ของคลาส
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        # คำนวณค่า similarity ระหว่าง embedding ของรูปภาพใหม่กับ embeddings ของคลาส
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/drug50cls_ViTModel'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# รูปภาพที่ต้องการทดสอบ
new_image_path = '/content/2.jpg'

# ค้นหาคลาสที่มีความคล้ายกันที่สุด 5 อันดับ
similar_classes = find_most_similar_classes(new_image_path, all_embeddings)

# แสดงผลลัพธ์
print("Top 5 most similar classes:")
for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
    print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

Top 5 most similar classes:
1. Class: Amlopine5mg, Similarity Score: 0.804720407185873
2. Class: Amlopine10mg, Similarity Score: 0.6334852880845288
3. Class: Blopress16mg, Similarity Score: 0.6282731733279555
4. Class: Betalol40mg, Similarity Score: 0.6118358029107045
5. Class: Prenolol100mg, Similarity Score: 0.5980021013161094


##640*640

###50cls

In [ ]:
# use VIT model to convert data
import pandas as pd
import numpy as np
import os
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import torch

# ใช้โมเดล ViT ของ Hugging Face ในการทำ embedding
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

def image_embedding(path):
    img = Image.open(path).convert('RGB').resize((640, 640))  # ปรับขนาดภาพให้เข้ากับ ViT
    inputs = processor(images=img, return_tensors="pt")
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    avg_embedding = last_hidden_states.mean(dim=1).squeeze().detach().numpy()  # ค่าเฉลี่ยของ hidden states

    curr_df = pd.DataFrame(avg_embedding).T
    return curr_df

def process_images_in_folder(folder_path, output_csv_path):
    pdEmbedded = pd.DataFrame()
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

    for idx, image_file in enumerate(image_files):
        image_path = os.path.join(folder_path, image_file)
        embedded = image_embedding(image_path)
        embedded['ID'] = image_file  # ใช้ชื่อไฟล์เป็น ID
        pdEmbedded = pd.concat([pdEmbedded, embedded], ignore_index=True)

    # สร้างโฟลเดอร์สำหรับ output_csv_path
    output_folder = os.path.dirname(output_csv_path)
    os.makedirs(output_folder, exist_ok=True)

    # บันทึก DataFrame ลงในไฟล์ CSV
    pdEmbedded.to_csv(output_csv_path, index=False)

# ฟังก์ชันสำหรับการวนลูปโฟลเดอร์
def process_all_folders(base_folder_path, output_base_folder):
    subfolders = [f.path for f in os.scandir(base_folder_path) if f.is_dir()]

    for subfolder in subfolders:
        subfolder_name = os.path.basename(subfolder)
        output_csv_path = os.path.join(output_base_folder, f"{subfolder_name}.csv")
        process_images_in_folder(subfolder, output_csv_path)
        print(f"Embedding data for {subfolder_name} has been saved to {output_csv_path}")

# ตัวอย่างการใช้งาน
base_folder_path = "/content/data 50 class add_augment - Copy"
output_base_folder = "/content/drug50cls_ViTModel_640"
os.makedirs(output_base_folder, exist_ok=True)  # สร้างโฟลเดอร์หลักถ้ายังไม่มี

process_all_folders(base_folder_path, output_base_folder)

###Cosine similarity

In [22]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import torch

# ใช้โมเดล ViT ของ Hugging Face ในการทำ embedding
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

# ย้ายโมเดลไปยัง GPU ถ้ามี
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def image_embedding(path):
    try:
        img = Image.open(path).convert('RGB').resize((224, 224))  # ปรับขนาดภาพให้เข้ากับ ViT
        inputs = processor(images=img, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        last_hidden_states = outputs.last_hidden_state
        avg_embedding = last_hidden_states.mean(dim=1).squeeze().cpu().detach().numpy()  # ค่าเฉลี่ยของ hidden states
        curr_df = pd.DataFrame(avg_embedding).T
        return curr_df
    except Exception as e:
        print(f"Error processing image {path}: {e}")
        return pd.DataFrame()

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            # ดรอปคอลัมน์ ID และเก็บเฉพาะเวกเตอร์ของคลาส
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        # คำนวณค่า similarity ระหว่าง embedding ของรูปภาพใหม่กับ embeddings ของคลาส
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/drug50cls_ViTModel_640'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# รูปภาพที่ต้องการทดสอบ
new_image_path = '/content/2.jpg'

# ค้นหาคลาสที่มีความคล้ายกันที่สุด 5 อันดับ
similar_classes = find_most_similar_classes(new_image_path, all_embeddings)

# แสดงผลลัพธ์
print("Top 5 most similar classes:")
for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
    print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

Top 5 most similar classes:
1. Class: Amlopine5mg, Similarity Score: 0.7900898666089824
2. Class: Amlopine10mg, Similarity Score: 0.6421481579057162
3. Class: Blopress16mg, Similarity Score: 0.6149773946927559
4. Class: Betalol40mg, Similarity Score: 0.6146524421261307
5. Class: Prenolol100mg, Similarity Score: 0.606150863561248


# โหลดข้อมูลเวกเตอร์

In [ ]:
# โหลดข้อมูลเวกเตอร์224*224
# Export Data drug50cls
# Define the folder you want to zip and download
import shutil

folder_to_download = '/content/drug50cls_ViTModel'

# Zip the folder
shutil.make_archive('/content/drive/MyDrive/drug/drug50cls_ViTModel', 'zip', folder_to_download)

# Download the zip file
from google.colab import files
files.download('/content/drive/MyDrive/drug/drug50cls_ViTModel.zip')

In [ ]:
# โหลดข้อมูลเวกเตอร์640*640
# Export Data drug50cls
# Define the folder you want to zip and download
import shutil

folder_to_download = '/content/drug50cls_ViTModel_640'

# Zip the folder
shutil.make_archive('/content/drive/MyDrive/drug/drug50cls_ViTModel_640', 'zip', folder_to_download)

# Download the zip file
from google.colab import files
files.download('/content/drive/MyDrive/drug/drug50cls_ViTModel_640.zip')